### Step 1: Install Dependencies

### Step 2: Setup and Imports

In [1]:
import os
import shutil
import numpy as np
import cv2
from pathlib import Path
from sklearn.model_selection import train_test_split
import albumentations as A
from ultralytics import YOLO
import yaml
import zipfile

BASE_DIR = Path.cwd()
DEFAULT_DATASET_DIRNAME = "laboratory.v1i.yolov11"
DATASET_ZIP_NAME = "laboratory.v1i.yolov11.zip"
DEFAULT_WEIGHTS_NAME = "yolo11n.pt"


def find_dataset_root(base: Path, dirname: str = DEFAULT_DATASET_DIRNAME) -> Path:
    candidates = [base / dirname, base]
    candidates.extend(p for p in base.glob("*/*") if p.name == dirname)
    for t in base.rglob("train"):
        images_dir = t / "images"
        labels_dir = t / "labels"
        if images_dir.exists() and labels_dir.exists():
            return t.parent
    for c in candidates:
        if (c / "train" / "images").exists() and (c / "train" / "labels").exists():
            return c
    return base


def find_weights(base: Path, filename: str = DEFAULT_WEIGHTS_NAME) -> Path:
    candidates = [base / filename]
    candidates.extend(base.glob(f"**/{filename}"))
    for c in candidates:
        if c.exists():
            return c
    return base / filename


def ensure_dataset_ready(base: Path) -> Path:
    zip_path = base / DATASET_ZIP_NAME
    root = find_dataset_root(base)
    if (root / "train" / "images").exists() and (root / "train" / "labels").exists():
        return root
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(base)
        return find_dataset_root(base)
    return base


ORIGINAL_DATA = ensure_dataset_ready(BASE_DIR)
SPLIT_DATA = BASE_DIR / "dataset_split"
AUGMENTED_DATA = BASE_DIR / "dataset_augmented"
MODEL_WEIGHTS = find_weights(BASE_DIR)

print(f"Base: {BASE_DIR}\nOriginal: {ORIGINAL_DATA}\nModel: {MODEL_WEIGHTS}")

if not (ORIGINAL_DATA / "train" / "images").exists():
    print("⚠ train/images not found")
if not (ORIGINAL_DATA / "train" / "labels").exists():
    print("⚠ train/labels not found")
if not MODEL_WEIGHTS.exists():
    print("⚠ MODEL_WEIGHTS not found")


Base: /home/usr4/Arsenii/sbor_data_pfur/lab_7
Original: /home/usr4/Arsenii/sbor_data_pfur/lab_7
Model: /home/usr4/Arsenii/sbor_data_pfur/lab_7/yolo11n.pt


### Step 3: Split Dataset (Train/Val/Test)

In [2]:
def split_dataset(
    source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15
):
    images_dir = source_dir / "train" / "images"
    labels_dir = source_dir / "train" / "labels"

    if not images_dir.exists():
        raise FileNotFoundError(f"Images folder not found: {images_dir}")
    if not labels_dir.exists():
        raise FileNotFoundError(f"Labels folder not found: {labels_dir}")

    for split in ["train", "val", "test"]:
        (output_dir / split / "images").mkdir(parents=True, exist_ok=True)
        (output_dir / split / "labels").mkdir(parents=True, exist_ok=True)

    image_files = sorted(
        [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".jpeg", ".png"))]
    )
    train_val, test = train_test_split(
        image_files, test_size=test_ratio, random_state=42
    )
    train, val = train_test_split(
        train_val, train_size=train_ratio / (train_ratio + val_ratio), random_state=42
    )

    splits = {"train": train, "val": val, "test": test}

    for split, files in splits.items():
        for img_file in files:
            label_file = img_file.rsplit(".", 1)[0] + ".txt"
            shutil.copy2(
                images_dir / img_file, output_dir / split / "images" / img_file
            )
            if (labels_dir / label_file).exists():
                shutil.copy2(
                    labels_dir / label_file, output_dir / split / "labels" / label_file
                )

    return splits


has_val_test = (ORIGINAL_DATA / "val" / "images").exists() and (
    ORIGINAL_DATA / "test" / "images"
).exists()
if has_val_test:
    SPLIT_DATA = ORIGINAL_DATA
    splits = {
        "train": sorted([p.name for p in (SPLIT_DATA / "train" / "images").glob("*")]),
        "val": sorted([p.name for p in (SPLIT_DATA / "val" / "images").glob("*")]),
        "test": sorted([p.name for p in (SPLIT_DATA / "test" / "images").glob("*")]),
    }
else:
    if SPLIT_DATA.exists():
        shutil.rmtree(SPLIT_DATA)
    splits = split_dataset(ORIGINAL_DATA, SPLIT_DATA)

print(
    f"TRAIN: {len(splits['train'])}, VAL: {len(splits['val'])}, TEST: {len(splits['test'])}"
)


TRAIN: 350, VAL: 75, TEST: 75


### Step 3.1: Convert polygon labels to YOLO bounding boxes

In [3]:
def polygon_to_bbox_line(line: str) -> str:
    parts = line.strip().split()
    if len(parts) < 5:
        raise ValueError("Invalid label line")
    coords = list(map(float, parts[1:]))
    if len(coords) % 2 != 0:
        raise ValueError("Invalid coordinates")
    xs, ys = coords[0::2], coords[1::2]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    width = x_max - x_min
    height = y_max - y_min
    return f"{parts[0]} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"


def convert_polygon_dir(src_dir: Path, dst_dir: Path) -> None:
    dst_dir.mkdir(parents=True, exist_ok=True)
    for src_file in src_dir.glob("*.txt"):
        with open(src_file, "r") as f_src, open(dst_dir / src_file.name, "w") as f_dst:
            for line in f_src:
                if line.strip():
                    f_dst.write(polygon_to_bbox_line(line) + "\n")


def convert_dataset_polygons_to_bboxes(base_dir: Path) -> None:
    for split in ["train", "val", "test"]:
        src_dir = base_dir / split / "labels"
        if src_dir.exists():
            convert_polygon_dir(src_dir, base_dir / split / "labels_bbox")


convert_dataset_polygons_to_bboxes(ORIGINAL_DATA)


### Step 4: Create data.yaml for YOLOv11

In [4]:
base_yaml_path = BASE_DIR / "data.yaml"
nc, names = 1, ["fruit"]
if base_yaml_path.exists():
    with open(base_yaml_path, "r") as f:
        existing = yaml.safe_load(f) or {}
    nc = existing.get("nc", nc)
    names = existing.get("names", names)

data_yaml = {
    "path": str(SPLIT_DATA),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": names,
}

yaml_path = SPLIT_DATA / "data.yaml"
with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)


### Step 5: Train YOLOv11 on Original Dataset

In [5]:
model = YOLO(str(MODEL_WEIGHTS))
results = model.train(
    data=str(yaml_path),
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0,
    project=str(BASE_DIR / "runs"),
    name="yolov11_original",
    save=True,
    verbose=False,
)


New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/usr4/Arsenii/sbor_data_pfur/lab_7/yolo11n.pt, m

### Step 6: Evaluate on Test Set

In [6]:
def find_best_run_model(run_name):
    candidates = [
        BASE_DIR / "runs" / "detect" / run_name / "weights" / "best.pt",
        BASE_DIR / "runs" / run_name / "weights" / "best.pt",
        BASE_DIR / "runs" / "detect" / run_name / "best.pt",
        BASE_DIR / "runs" / run_name / "best.pt",
    ]
    for p in candidates:
        if p.exists():
            return p
    runs_dir = BASE_DIR / "runs"
    if runs_dir.exists():
        for p in runs_dir.rglob("best.pt"):
            if run_name in str(p):
                return p
    return None


best_orig_path = find_best_run_model("yolov11_original")
if not best_orig_path:
    raise FileNotFoundError("best.pt for yolov11_original not found")

best_model = YOLO(str(best_orig_path))
test_results = best_model.val(
    data=str(yaml_path), split="test", imgsz=640, batch=16, device=0
)
print(f"mAP50: {test_results.results_dict.get('metrics/mAP50', 0):.4f}")


Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1465.3±745.3 MB/s, size: 18.1 KB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1465.3±745.3 MB/s, size: 18.1 KB)
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels... 75 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 3.5Kit/s 0.0s
val: New cache created: /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels... 75 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 3.5Kit/s 0.0s
val: New cache created: /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache
                 Class     Images  Instances      Box(P   

### Step 7: Data Augmentation using Albumentations

In [7]:
def augment_dataset(source_dir, output_dir, augmentation_factor=2):
    for split in ["train", "val", "test"]:
        (output_dir / split / "images").mkdir(parents=True, exist_ok=True)
        (output_dir / split / "labels").mkdir(parents=True, exist_ok=True)

    for split in ["val", "test"]:
        src_img = source_dir / split / "images"
        src_lbl = source_dir / split / "labels"
        dst_img = output_dir / split / "images"
        dst_lbl = output_dir / split / "labels"
        if src_img.exists():
            for f in src_img.glob("*"):
                shutil.copy2(f, dst_img / f.name)
            for f in src_lbl.glob("*"):
                shutil.copy2(f, dst_lbl / f.name)

    transform = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.Rotate(limit=20, p=0.4),
            A.GaussNoise(p=0.2),
            A.Blur(blur_limit=3, p=0.15),
        ],
        bbox_params=A.BboxParams(
            format="yolo", min_visibility=0.0, label_fields=["class_ids"]
        ),
    )

    src_train_img = source_dir / "train" / "images"
    src_train_lbl = source_dir / "train" / "labels"
    dst_train_img = output_dir / "train" / "images"
    dst_train_lbl = output_dir / "train" / "labels"

    image_files = sorted(list(src_train_img.glob("*")))
    counter = 0

    for img_path in image_files:
        image = cv2.imread(str(img_path))
        if image is None:
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        label_path = src_train_lbl / (img_path.stem + ".txt")

        bboxes, class_ids = [], []
        if label_path.exists():
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                    cls = int(float(parts[0]))
                    vals = [float(x) for x in parts[1:5]]

                    if max(vals) > 1.0:
                        x1, y1, x2, y2 = vals
                        if x2 > x1 and y2 > y1:
                            xc = ((x1 + x2) / 2.0) / w
                            yc = ((y1 + y2) / 2.0) / h
                            bw = (x2 - x1) / w
                            bh = (y2 - y1) / h
                        else:
                            xc, yc, bw, bh = (
                                vals[0] / w,
                                vals[1] / h,
                                vals[2] / w,
                                vals[3] / h,
                            )
                    else:
                        xc, yc, bw, bh = vals

                    xc = np.clip(xc, 0.0, 1.0)
                    yc = np.clip(yc, 0.0, 1.0)
                    bw = np.clip(bw, 0.001, 1.0)
                    bh = np.clip(bh, 0.001, 1.0)

                    x1_norm = max(0.0, xc - bw / 2)
                    y1_norm = max(0.0, yc - bh / 2)
                    x2_norm = min(1.0, xc + bw / 2)
                    y2_norm = min(1.0, yc + bh / 2)

                    if x2_norm <= x1_norm or y2_norm <= y1_norm:
                        continue

                    bbox_norm = [
                        (x1_norm + x2_norm) / 2,
                        (y1_norm + y2_norm) / 2,
                        x2_norm - x1_norm,
                        y2_norm - y1_norm,
                    ]
                    bboxes.append(bbox_norm)
                    class_ids.append(cls)

        shutil.copy2(img_path, dst_train_img / img_path.name)
        if label_path.exists():
            shutil.copy2(label_path, dst_train_lbl / label_path.name)
        counter += 1

        for a in range(augmentation_factor):
            if bboxes:
                transformed = transform(image=image, bboxes=bboxes, class_ids=class_ids)
                aug_image = transformed["image"]
                aug_bboxes = transformed["bboxes"]
                aug_class_ids = transformed.get("class_ids", [])
            else:
                aug_image = A.Compose([A.HorizontalFlip(p=0.5)])(image=image)["image"]
                aug_bboxes, aug_class_ids = [], []

            cv2.imwrite(
                str(dst_train_img / f"{img_path.stem}_aug_{a}{img_path.suffix}"),
                cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR),
            )

            with open(dst_train_lbl / f"{img_path.stem}_aug_{a}.txt", "w") as f:
                for cid, bbox in zip(aug_class_ids, aug_bboxes):
                    xc, yc, bw, bh = bbox
                    xc = np.clip(xc, 0.0, 1.0)
                    yc = np.clip(yc, 0.0, 1.0)
                    bw = np.clip(bw, 0.001, 1.0)
                    bh = np.clip(bh, 0.001, 1.0)
                    if bw > 0 and bh > 0:
                        f.write(f"{int(cid)} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

            counter += 1

    return counter


if AUGMENTED_DATA.exists():
    shutil.rmtree(AUGMENTED_DATA)

aug_count = augment_dataset(SPLIT_DATA, AUGMENTED_DATA, augmentation_factor=2)


### Step 8: Create data.yaml for Augmented Dataset

In [8]:
data_yaml_aug = {
    "path": str(AUGMENTED_DATA),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": names,
}

yaml_path_aug = AUGMENTED_DATA / "data.yaml"
with open(yaml_path_aug, "w") as f:
    yaml.dump(data_yaml_aug, f, default_flow_style=False)


### Step 9: Train YOLOv11 on Augmented Dataset

In [9]:
model_aug = YOLO(str(MODEL_WEIGHTS))
results_aug = model_aug.train(
    data=str(yaml_path_aug),
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0,
    project=str(BASE_DIR / "runs"),
    name="yolov11_augmented",
    save=True,
    verbose=False,
)


New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_augmented/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/usr4/Arsenii/sbor_data_pfur/lab_7/yolo11n.p

### Step 10: Compare Results

In [10]:
best_model_original_path = find_best_run_model("yolov11_original")
best_model_augmented_path = find_best_run_model("yolov11_augmented")

if not best_model_original_path or not best_model_augmented_path:
    raise FileNotFoundError(f"Missing best.pt files")

best_model_original = YOLO(str(best_model_original_path))
best_model_augmented = YOLO(str(best_model_augmented_path))

test_results_original = best_model_original.val(
    data=str(yaml_path), split="test", imgsz=640, batch=16, device=0
)
test_results_augmented = best_model_augmented.val(
    data=str(yaml_path_aug), split="test", imgsz=640, batch=16, device=0
)

orig_map = test_results_original.results_dict.get("metrics/mAP50", 0)
aug_map = test_results_augmented.results_dict.get("metrics/mAP50", 0)

print(f"Original mAP50:  {orig_map:.4f}")
print(f"Augmented mAP50: {aug_map:.4f}")
if orig_map > 0:
    print(f"Change: {(aug_map - orig_map) / orig_map * 100:+.2f}%")


Ultralytics 8.3.235 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24219MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 192.2±34.0 MB/s, size: 13.5 KB)
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache... 75 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 26.2Mit/s 0.0sYOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 192.2±34.0 MB/s, size: 13.5 KB)
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache... 75 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 250.1Kit/s 0.0s
val: Scanning /home/usr4/Arsenii/sbor_data_pfur/lab_7/dataset_split/test/labels.cache... 75 images, 16 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 75/75 250.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      m

### Step 11: Summary and Model Export